# Machine Learning Project

In this project, we will walk through the full data science life cycle, from data cleaning and feature selection to machine learning. We will focus on credit modelling, a well known data science problem that focuses on modeling a borrower's [credit risk](https://en.wikipedia.org/wiki/Credit_risk). Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from [Lending Club](https://www.lendingclub.com/). Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. We can read more about their marketplace [here](https://www.lendingclub.com/public/how-peer-lending-works.action).

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. We can read more about the interest rate that Lending Club assigns [here](https://www.lendingclub.com/public/borrower-rates-and-fees.action). Lending Club also tries to verify each piece of information the borrower provides but it can't always verify all of the information (usually for regulation reasons).

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a [grade](https://www.lendingclub.com/public/rates-and-fees.action) according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiving a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the [origination fee](https://help.lendingclub.com/hc/en-us/articles/214501207-What-is-the-origination-fee-) that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off before they see a return in money. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition to the requested amount. Many loans aren't completely paid off on time, however, and some borrowers **default** on the loan.

Here's a diagram from [Bible Money Matters](http://www.biblemoneymatters.com/introduction-to-peer-to-peer-lending-signing-up-to-use-lending-club/) that sums up the process:

![](http://cdn.biblemoneymatters.com/wp-content/uploads/2009/08/how-social-lending-works.jpg)

While Lending Club has to be extremely savvy and rigorous with their credit modelling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. While at first, we may wonder why investors would put money into anything but low interest loans. The incentive investors have to back higher interest loans is, well, the higher interest! If investors believe the borrower can pay back the loan, even if he or she has a weak financial history, then investors can make more money through the larger additional amount the borrower has to pay.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this course, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need to first understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

## Introduction to the Data 

Lending Club releases data for all of the approved and declined loan applications periodically [on their website](https://www.lendingclub.com/info/download-data.action). We can select a few different year ranges to download the datasets (in CSV format) for both approved and declined loans.

We'll also find a **data dictionary** (in XLS format) which contains information on the different column names towards the bottom of the page. Here's a link to the data dictionary file hosted on [Google Drive](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit).

Before diving into the datasets themselves, let's get familiar with the data dictionary. The **LoanStats** sheet describes the approved loans datasets and the **RejectStats* describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

+ *Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?*

Before we can start doing machine learning, we need to define what features we want to use and which column represents the target column we want to predict. Let's start by reading in the dataset and exploring it.

## Reading into Pandas

In this mission, we'll focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current and still being paid off.

In [1]:
import pandas as pd

loans_2007 = pd.read_csv("loans_2007.csv", low_memory=False)
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

## First Group of Columns

The Dataframe contains many columns and can be cumbersome to try to explore all at once. Let's break up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. As you understand each feature, you want to pay attention to any features that:

+ leak information from the future (after the loan has already been funded)
+ don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
+ formatted poorly and need to be cleaned up
+ require more data or a lot of processing to turn into a useful feature
+ contain redundant information

We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans. We encourage you to spend as much time as you need to understand each column, because a poor understanding could cause you to make mistakes in the data analysis and modeling process. As you go through the dictionary, keep in mind that we need to select one of the columns as the target column we want to use when we move on to the machine learning phase.

Let's focus on just columns that we need to remove from consideration. Then, we can circle back and further dissect the columns we decided to keep.

In [2]:
loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

## Second Group of Columns

Let's now look at the next 18 columns.

Within this group of columns, we need to drop the following columns:

+ `zip_code`: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
+ `out_prncp`: leaks data from the future, (after the loan already started to be paid off)
+ `out_prncp_inv`: also leaks data from the future, (after the loan already started to be paid off)
+ `total_pymnt`: also leaks data from the future, (after the loan already started to be paid off)
+ `total_pymnt_inv`: also leaks data from the future, (after the loan already started to be paid off)
+ `total_rec_prncp`: also leaks data from the future, (after the loan already started to be paid off)

The `out_prncp` and `out_prncp_inv` both describe the outstanding principal amount for a loan, which is the remaining amount the borrower still owes. These 2 columns as well as the `total_pymnt` column describe properties of the loan after it's fully funded and started to be paid off. This information isn't available to an investor before the loan is fully funded and we don't want to include it in our model.

Let's go ahead and remove these columns from the Dataframe.

In [3]:
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

## Third Group of Features

Let's now move on to the last group of features.

In the last group of columns, we need to drop the following columns:

`total_rec_int`: leaks data from the future, (after the loan already started to be paid off),
`total_rec_late_fee`: also leaks data from the future, (after the loan already started to be paid off),
`recoveries`: also leaks data from the future, (after the loan already started to be paid off),
`collection_recovery_fee`: also leaks data from the future, (after the loan already started to be paid off),
`last_pymnt_d`: also leaks data from the future, (after the loan already started to be paid off),
`last_pymnt_amnt`: also leaks data from the future, (after the loan already started to be paid off).

All of these columns leak data from the future, meaning that they're describing aspects of the loan after it's already been fully funded and started to be paid off by the borrower.

In [4]:
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)

print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

## Target Column

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

We should use the `loan_status` column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [5]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

## Binary Classification 

There are 8 different possible values for the `loan_status` column. We can read about most of the different loan statuses [on the Lending Clube website](https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-). The two values that start with "Does not meet the credit policy" aren't explained unfortunately. A quick Google search takes us to explanations from the lending community [here](http://www.lendacademy.com/forum/index.php?topic=2427.msg20813#msg20813).

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the `Fully Paid` and `Charged Off` values describe the final outcome of the loan. The other values describe loans that are still ongoing and where the jury is still out on if the borrower will pay back the loan on time or not. While the `Default` status resembles the `Charged Off` status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. We can read about the difference [here](http://kb.lendingclub.com/investor/articles/Investor/What-is-the-difference-between-a-loan-that-is-in-default-and-a-loan-that-has-been-charged-off).

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a **binary classification** one. Let's remove all the loans that don't contain either `Fully Paid` and `Charged Off` as the loan's status and then transform the `Fully Paid` values to `1` for the positive case and the `Charged Off` values to `0` for the negative case. 

Lastly, one thing we need to keep in mind is the **class imbalance** between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. There are a few different ways to tackle this class imbalance, which we'll explore later.

In [6]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') | (loans_2007['loan_status'] == 'Charged Off')]

status_replace = {'Fully Paid':1,
                'Charged Off':0
               }
loans_2007 = loans_2007.replace(status_replace)

## Removing Single Value Columns

Let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application. In addition, removing these columns will reduce the number of columns we'll need to explore further in the next mission.

In [7]:
orig_columns = loans_2007.columns
drop_columns = []

for col in loans_2007.columns:
    
    non_null = loans_2007[col].dropna()
    unique_non_null = non_null.unique()
    
    if len(unique_non_null) == 1:
        
        drop_columns.append(col)
        
loans = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


It looks we were able to remove 9 more columns since they only contained 1 unique value.

## Handling Missing Values

Let's compute the number of missing values and come up with a strategy for handling them. Then, we'll focus on the categorical columns.

In [8]:
null_counts = loans.isnull().sum()
print(null_counts[null_counts>0])

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


While most of the columns have no missing values, two columns have fifty or less rows with missing values, and two columns, `emp_length` and `pub_rec_bankruptcies`, contain a relatively high amount of missing values.

Domain knowledge tells us that employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values.

Let's inspect the values of the column `pub_rec_bankruptcies`.

In [9]:
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


We see that this column offers very little variability, nearly 94% of values are in the same category. It probably won't have much predictive value. Let's drop it. In addition, we'll remove the remaining rows containing null values.

This means that we'll keep the following columns and just remove rows containing missing values for them:

+ `emp_length`
+ `title`
+ `revol_util`
+ `last_credit_pull_d`

and `drop the pub_rec_bankruptcies` column entirely.

Let's use the strategy of removing the `pub_rec_bankruptcies` column first, then removing all rows containing any missing values at all to cover both of these cases. This way, we only remove the rows containing missing values for the `emp_length`, `title` and `revol_util columns`, but not the `pub_rec_bankruptcies` column.

In [10]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


## Text Columns

While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth.

Let's select just the object columns then display a sample row to get a better sense of how the values in each column are formatted.

In [11]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


## Converting Text Columns

Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

+ `home_ownership`: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
+ `verification_status`: indicates if income was verified by Lending Club,
+ `emp_length`: number of years the borrower was employed upon time of application,
+ `term`: number of payments on the loan, either 36 or 60,
+ `addr_state`: borrower's state of residence,
+ `purpose`: a category provided by the borrower for the loan request,
+ `title`: loan title provided by the borrower,

There are also some columns that represent numeric values, that need to be converted:

+ `int_rate`: interest rate of the loan in %,
+ `revol_util`: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit, read more here.

Based on the first row's values for `purpose` and `title`, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

+ `earliest_cr_line`: The month the borrower's earliest reported credit line was opened,
+ `last_credit_pull_d`: The most recent month Lending Club pulled credit for this loan.

Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

In [12]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for col in cols:
    print(loans[col].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
UT     249
KS     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY      76
AK      76
SD      60
VT  

## The Reason for the Loan 

The home_ownership, `verification_status`, `emp_length`, `term`, and `addr_state columns` all contain multiple discrete values. We should clean the `emp_length` column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

First, let's look at the unique value counts for the `purpose` and `title` columns to understand which column we want to keep.

In [13]:
print(loans["title"].value_counts())
print(loans["purpose"].value_counts())

Debt Consolidation                      2068
Debt Consolidation Loan                 1599
Personal Loan                            624
Consolidation                            488
debt consolidation                       466
                                        ... 
School/Credit Card                         1
Second Mortgage and Moving Expenses        1
Refinance Credit Card at Better Rate       1
Credit Card and Debt Consolidation         1
Tofiga7                                    1
Name: title, Length: 18881, dtype: int64
debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64


## Categorical Columns

The `home_ownership`, `verification_status`, `emp_length`, and `term` columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the `purpose` and `title` columns do contain overlapping information but we'll keep the `purpose` column since it contains a few discrete values. In addition, the `title` column has data quality issues since many of the values are repeated with slight modifications (e.g. `Debt Consolidation` and `Debt Consolidation Loan` and `debt consolidation`).

We can use the following mapping to clean the `emp_length` column:

+ "10+ years": 10
+ "9 years": 9
+ "8 years": 8
+ "7 years": 7
+ "6 years": 6
+ "5 years": 5
+ "4 years": 4
+ "3 years": 3
+ "2 years": 2
+ "1 year": 1
+ "< 1 year": 0
+ "n/a": 0

We erred on the side of being conservative with the `10+ years`, `< 1 year` and `n/a` mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

Lastly, the `addr_state` column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

In [14]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1)

loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')

loans = loans.replace(mapping_dict)

## Dummy Variables

Let's now encode the `home_ownership`, `verification_status`, `purpose`, and `term` columns as dummy variables so we can use them in our model.

In [15]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            37675 non-null  float64
 1   int_rate                             37675 non-null  float64
 2   installment                          37675 non-null  float64
 3   emp_length                           37675 non-null  int64  
 4   annual_inc                           37675 non-null  float64
 5   loan_status                          37675 non-null  int64  
 6   dti                                  37675 non-null  float64
 7   delinq_2yrs                          37675 non-null  float64
 8   inq_last_6mths                       37675 non-null  float64
 9   open_acc                             37675 non-null  float64
 10  pub_rec                              37675 non-null  float64
 11  revol_bal                   

## Picking an Error Metric

Before we dive into predicting `loan_status` with machine learning, let's go back to our first steps when we started cleaning the Lending Club dataset. You may recall the original question we wanted to answer:

+ *Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?*

We established that this is a binary classification problem in the first mission of this course, and we converted the `loan_status` column to `0s` and `1s` as a result. Before diving in and selecting an algorithm to apply to the data, we should select an error metric.

An error metric will help us figure out when our model is performing well, and when it's performing poorly. To tie error metrics all the way back to the original question we wanted to answer, let's say we're using a machine learning model to predict whether or not we should fund a loan on the Lending Club platform. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

Here's a diagram to simplify the concepts:

| loan_status actual | predictions | error metric |
| :-: | :-: | :-: |
| 0 | 1 | False Positive |
| 1 | 1 | True Positive |
| 0 | 0 | True Negative |
| 1 | 0 | False Negative |

In the `loan_status` and `prediction columns`, a `0` means that the loan wouldn't be paid off on time, and a `1` means that it would.

Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

## Class Imbalance

We mentioned earlier that there is a significant class imbalance in the `loan_status` column. There are `6` times as many loans that were paid off on time (`1`), than loans that weren't paid off on time (`0`). This causes a major issue when we use accuracy as a metric. This is because due to the class imbalance, a classifier can predict `1` for every row, and still have high accuracy. Here's a diagram that illustrates the concept:

| loan_status actual | predictions |
| :-: | :-: |
| 0 | 1 |
| 1 | 1 |
| 1 | 1 |
| 1 | 1 |
| 1 | 1 |
| 1 | 1 | 
| 1 | 1 |

In the above diagram, our predictions are `85.7%` accurate -- we've correctly identified `loan_status` in `85.7%` of cases. However, we've done this by predicting `1` for every row. What this means is that we'll actually lose money. Let's say we loan out `1000` dollars on average to each borrower. Each borrower pays us `10%` interest back. So we make a projected profit of `100` dollars on each loan. In the above diagram, we'd actually lose money:

| loan_status actual | predictions | profit/loss |
| :-: | :-: | :- |
| 0 | 1 | -1000 |
| 1 | 1 | 100 |
| 1 | 1 | 100 |
| 1 | 1 | 100 |
| 1 | 1 | 100 |
| 1 | 1 | 100 |
| 1 | 1 | 100 |

As We can see, we made `600` dollars in interest from the borrowers that paid us back, but we lost `1000` dollars on the one borrower who never paid us back, so we actually ended up losing `400` dollars overall, even though our model is technically accurate.

This is why it's important to always be aware of imbalanced classes in machine learning models, and to adjust your error metric accordingly. In this case, we don't want to use accuracy, and should instead use metrics that tell us the number of false positives and false negatives.

This means that we should optimize for:

+ high [recall](https://en.wikipedia.org/wiki/Precision_and_recall#Recall) (true positive rate)
+ low [fall-out](https://en.wikipedia.org/wiki/Information_retrieval#Fall-out) (false positive rate)

We can calculate false positive rate and true positive rate, using the numbers of true positives, true negatives, false negatives, and false positives.

False positive rate is the number of false positives divided by the number of false positives plus the number of true negatives. This divides all the cases where we thought a loan would be paid off but it wasn't by all the loans that weren't paid off.

True positive rate is the number of true positives divided by the number of true positives plus the number of false negatives. This divides all the cases where we thought a loan would be paid off and it was by all the loans that were paid off.

We can write these out as mathematical formulas as well:

\begin{equation}
FPR=\dfrac{\text{False Positives}}{\text{False Positives} + \text{True Negatives}}
\end{equation}

\begin{equation}
TPR=\dfrac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
\end{equation}

Simple english ways to think of each term are:

+ False Positive Rate: "the percentage of the loans that shouldn't be funded that I would fund".
+ True Positive Rate: "the percentage of loans that should be funded that I would fund".

Generally, if we want to reduce false positive rate, true positive rate will also go down. This is because if we want to reduce the risk of false positives, we wouldn't think about funding riskier loans in the first place.

## Logistic Regression

A good first algorithm to apply to binary classification problems is [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression), for the following reasons:

+ it's quick to train and we can iterate more quickly,
+ it's less prone to overfitting than more complex models like decision trees,
+ it's easy to interpret.

In [17]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

features = loans.drop('loan_status', axis=1)
target = loans['loan_status']

lr.fit(features, target)
predictions = lr.predict(features)

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

## Cross Validation

While we generated predictions in the last screen, those predictions were overfit. They were overfit because we generated predictions using the same data that we trained our model on. When we use this to evaluate error, we get an unrealistically high depiction of how accurate the algorithm is, because it already "knows" the correct answers. This is like asking someone to memorize a bunch of physics equations, then asking them to plug numbers into the equations. They can tell you the right answer, but they can't explain a concept that they haven't already memorized an equation for.

In order to get a realistic depiction of the accuracy of the model, let's perform [k-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation). We can use the [cross_val_predict()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) function from the [sklearn.model_selection](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) package.

Once we have cross validated predictions, we can compute true positive rate and false positive rate.

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(tpr)
print(fpr)

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9984596224436287
0.9986179664363277


C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Penalizing the Classifier

As you can see from the last screen, our `fpr` and `tpr` are around what we'd expect if the model was predicting all ones. We can look at the first few rows of `predictions` to confirm:

In [19]:
print(predictions.head(10))

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int64


Unfortunately, even though we're not using accuracy as an error metric, the classifier is, and it isn't accounting for the imbalance in the classes. There are a few ways to get a classifier to correct for imbalanced classes. The two main ways are:

+ Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
+ Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.

We'll look into oversampling and undersampling first. They involve taking a sample that contains equal numbers of rows where `loan_status` is `0`, and where `loan_status` is `1`. This way, the classifier is forced to make actual predictions, since predicting all `1s` or all `0s` will only result in `50%` accuracy at most.

The downside of this technique is that since it has to preserve an equal ratio, you have to either:

+ Throw out many rows of data. If we wanted equal numbers of rows where `loan_status` is `0` and where `loan_status` is `1`, one way we could do that is to delete rows where `loan_status` is `1`.
+ Copy rows multiple times. One way to equalize the `0s` and `1s` is to copy rows where `loan_status` is `0`.
+ Generate fake data. One way to equalize the `0s` and `1s` is to generate new rows where `loan_status` is `0`.

Unfortunately, none of these techniques are especially easy. The second method we mentioned earlier, telling the classifier to penalize certain rows more, is actually much easier to implement using scikit-learn.

We can do this by setting the `class_weight` parameter to `balanced` when creating the LogisticRegression instance. This tells scikit-learn to penalize the misclassification of the minority class during the training process. The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where `loan_status` is `0`. This lowers accuracy when `loan_status` is `1`, but raises accuracy when `loan_status` is `0`.

By setting the `class_weight` parameter to `balanced`, the penalty is set to be inversely proportional to the class frequencies. We can read more about the parameter [here](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn-linear-model-logisticregression). This would mean that for the classifier, correctly classifying a row where `loan_status` is `0` is `6` times more important than correctly classifying a row where `loan_status` is `1`.

We can repeat the cross validation procedure we performed in the last screen, but with the `class_weight` parameter set to `balanced`.

In [20]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(class_weight='balanced')
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(tpr)
print(fpr)

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-r

0.5465718405873099
0.5237907206317868


C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Manual Penalties

We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate. Our true positive rate is now around `55%`, and our false positive rate is around `52%`. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever decide to fund `55%` of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting `class_weight` to balanced will automatically set a penalty based on the number of `1s` and `0s` in the column, we can also set a manual penalty. In the last screen, the penalty scikit-learn imposed for misclassifying a `0` would have been around `5.89` (since there are `5.89` times as many `1s` as `0s`).

We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a dictionary of penalty values to the `class_weight` parameter.

In [21]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(tpr)
print(fpr)

C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.1704902988987939
0.16722606120434352


C:\Users\Utente\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Random Forests

It looks like assigning manual penalties lowered the false positive rate to `17%`, and thus lowered our risk. Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs.

While we could tweak the penalties further, it's best to move to trying a different model right now, for larger potential false positive rate gains. We can always loop back and iterate on the penalties more later.

Let's try a more complex algorithm, random forest. We learned about random forests in a previous mission, and constructed our own model. Random forests are able to work with nonlinear data, and learn complex conditionals. Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with `loan_status`.

We can use the [RandomForestClassifer](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) class from scikit-learn to do this.

In [22]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=1, class_weight='balanced')
rfc.fit(features, target)
predictions = cross_val_predict(rfc, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(tpr)
print(fpr)

0.996722600943891
0.9982230997038499


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the `1` class, and still mostly predicting `1s`. We could fix this by applying a harsher penalty for misclassifications of `0s`.

Ultimately, our best model had a false positive rate of nearly `17%`, and a true positive rate of `17%` as well. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from `17%` of borrowers defaulting, and that the pool of `17%` of borrowers is large enough to make enough interest money to offset the losses.

If we had randomly picked loans to fund, borrowers would have defaulted on `14.5%` of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

+ We can tweak the penalties further.
+ We can try models other than a random forest and logistic regression.
+ We can use some of the columns we discarded to generate better features.
+ We can ensemble multiple models to get more accurate predictions.
+ We can tune the parameters of the algorithm to achieve higher performance.